In [ ]:
# SwinIR Super Resolution with Results Saving
# This script runs SwinIR for super resolution, displays and saves the results

# Clone SwinIR repository
!git clone https://github.com/JingyunLiang/SwinIR.git
!pip install timm -q

# Download the pre-trained models (both regular and large versions)
!mkdir -p experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth -P experiments/pretrained_models

# Setup folders for input and output
import os
import glob
import shutil
from google.colab import files
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Define folders for input and output
upload_folder = 'input_images'
result_folder = 'results'
saved_results_folder = 'saved_results'  # New folder specifically for saved results

# Clean up existing folders
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
if os.path.isdir(saved_results_folder):
    shutil.rmtree(saved_results_folder)

# Create folders
os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(os.path.join(result_folder, 'SwinIR'), exist_ok=True)
os.makedirs(os.path.join(result_folder, 'SwinIR_large'), exist_ok=True)
os.makedirs(saved_results_folder, exist_ok=True)
os.makedirs(os.path.join(saved_results_folder, 'SwinIR'), exist_ok=True)
os.makedirs(os.path.join(saved_results_folder, 'SwinIR_large'), exist_ok=True)

# Option to process large images in patches (set to True if you encounter memory issues)
test_patch_wise = False
print(f"Processing large images with patches: {test_patch_wise}")
print("Upload your images when prompted...")

# Upload images
uploaded = files.upload()
for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'Moving {filename} to {dst_path}')
    shutil.move(filename, dst_path)

# Function to read images with RGB conversion
def imread(img_path):
    """Read an image from path and convert it from BGR to RGB"""
    if not os.path.exists(img_path):
        print(f"Warning: Image not found: {img_path}")
        return None

    img = cv2.imread(img_path)
    if img is None:
        print(f"Warning: Failed to read image: {img_path}")
        return None

    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Function to save images in high quality
def save_image(img, save_path):
    """Save image in RGB format with high quality"""
    # Convert RGB to BGR for OpenCV
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Use lossless compression
    params = [cv2.IMWRITE_PNG_COMPRESSION, 0]  # 0 = no compression (highest quality)

    # Save the image
    result = cv2.imwrite(save_path, img_bgr, params)

    if result:
        print(f"Successfully saved image to: {save_path}")
    else:
        print(f"Failed to save image to: {save_path}")

    return result

# Function to display results
def display_results(img_input, swinir_img, swinir_large_img):
    """Display input and output images side by side"""
    fig = plt.figure(figsize=(18, 6))

    # Input image
    ax1 = fig.add_subplot(1, 3, 1)
    plt.title('Input image', fontsize=16)
    ax1.axis('off')
    ax1.imshow(img_input)

    # SwinIR output
    ax2 = fig.add_subplot(1, 3, 2)
    plt.title('SwinIR output', fontsize=16)
    ax2.axis('off')
    if swinir_img is not None:
        ax2.imshow(swinir_img)

    # SwinIR-Large output
    ax3 = fig.add_subplot(1, 3, 3)
    plt.title('SwinIR-Large output', fontsize=16)
    ax3.axis('off')
    if swinir_large_img is not None:
        ax3.imshow(swinir_large_img)

    plt.tight_layout()
    plt.show()

# Run SwinIR (regular version)
print("\nRunning SwinIR (Medium model)...")
if test_patch_wise:
    !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq {upload_folder} --scale 4 --tile 800
else:
    !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq {upload_folder} --scale 4

# Move results to our results folder
if os.path.exists('results/swinir_real_sr_x4'):
    for file in glob.glob('results/swinir_real_sr_x4/*'):
        shutil.copy(file, os.path.join(result_folder, 'SwinIR'))

# Run SwinIR-Large (large version)
print("\nRunning SwinIR-Large (Large model)...")
if test_patch_wise:
    !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth --folder_lq {upload_folder} --scale 4 --large_model --tile 640
else:
    !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth --folder_lq {upload_folder} --scale 4 --large_model

# Move results to our results folder
if os.path.exists('results/swinir_real_sr_x4_large'):
    for file in glob.glob('results/swinir_real_sr_x4_large/*'):
        # Rename files to avoid naming conflicts
        filename = os.path.basename(file)
        new_filename = filename.replace('SwinIR.png', 'SwinIR_large.png')
        shutil.copy(file, os.path.join(result_folder, 'SwinIR_large', new_filename))

# Process and save the results
print("\nSaving and displaying results...")
input_files = sorted(glob.glob(os.path.join(upload_folder, '*')))

for input_path in input_files:
    filename = os.path.basename(input_path)
    basename, ext = os.path.splitext(filename)

    # Get input image
    input_img = imread(input_path)
    if input_img is None:
        print(f"Skipping {input_path} - could not read image")
        continue

    # Get SwinIR output
    swinir_path = os.path.join(result_folder, 'SwinIR', f"{basename}_SwinIR.png")
    swinir_img = imread(swinir_path)

    # Get SwinIR-Large output
    swinir_large_path = os.path.join(result_folder, 'SwinIR_large', f"{basename}_SwinIR_large.png")
    swinir_large_img = imread(swinir_large_path)

    print(f"\nResults for {filename}:")

    # Save the enhanced images to the dedicated saved_results folder
    # Save the original input image too for reference
    save_image(input_img, os.path.join(saved_results_folder, f"{basename}_original{ext}"))

    if swinir_img is not None:
        # Save SwinIR result
        save_path = os.path.join(saved_results_folder, 'SwinIR', f"{basename}_SwinIR_enhanced.png")
        save_image(swinir_img, save_path)

    if swinir_large_img is not None:
        # Save SwinIR-Large result
        save_path = os.path.join(saved_results_folder, 'SwinIR_large', f"{basename}_SwinIR_large_enhanced.png")
        save_image(swinir_large_img, save_path)

    # Display the results
    display_results(input_img, swinir_img, swinir_large_img)

    # Show image dimensions
    print(f"Input size: {input_img.shape[1]}×{input_img.shape[0]}")
    if swinir_img is not None:
        print(f"SwinIR output size: {swinir_img.shape[1]}×{swinir_img.shape[0]}")
    if swinir_large_img is not None:
        print(f"SwinIR-Large output size: {swinir_large_img.shape[1]}×{swinir_large_img.shape[0]}")

    # Calculate resolution increase
    if swinir_img is not None:
        input_pixels = input_img.shape[0] * input_img.shape[1]
        output_pixels = swinir_img.shape[0] * swinir_img.shape[1]
        print(f"Resolution increased by: {output_pixels/input_pixels:.2f}x")

    # Add a separator for clarity
    print("-" * 50)

# Create comparison images (before/after side by side)
print("\nCreating comparison images...")
os.makedirs(os.path.join(saved_results_folder, 'comparisons'), exist_ok=True)

for input_path in input_files:
    filename = os.path.basename(input_path)
    basename, ext = os.path.splitext(filename)

    # Get input image
    input_img = imread(input_path)
    if input_img is None:
        continue

    # Get SwinIR large output (using large version for comparison)
    swinir_large_path = os.path.join(result_folder, 'SwinIR_large', f"{basename}_SwinIR_large.png")
    swinir_large_img = imread(swinir_large_path)

    if swinir_large_img is not None:
        # Create a side-by-side comparison
        h1, w1 = input_img.shape[:2]
        h2, w2 = swinir_large_img.shape[:2]

        # Resize input image to match the height of the enhanced image for fair comparison
        aspect_ratio = w1 / h1
        new_h = h2
        new_w = int(new_h * aspect_ratio)
        input_img_resized = cv2.resize(input_img, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)

        # Create the comparison image
        comparison = np.zeros((h2, w2 + new_w, 3), dtype=np.uint8)
        comparison[:, :new_w] = input_img_resized
        comparison[:, new_w:] = swinir_large_img

        # Add a vertical line between the images
        comparison[:, new_w-1:new_w+1] = [255, 255, 255]  # White line

        # Add labels
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(comparison, "Original (Resized)", (10, 30), font, 1, (255, 255, 255), 2)
        cv2.putText(comparison, "SwinIR Enhanced", (new_w + 10, 30), font, 1, (255, 255, 255), 2)

        # Save the comparison
        comparison_path = os.path.join(saved_results_folder, 'comparisons', f"{basename}_comparison.png")
        comparison_bgr = cv2.cvtColor(comparison, cv2.COLOR_RGB2BGR)
        cv2.imwrite(comparison_path, comparison_bgr)
        print(f"Saved comparison image: {comparison_path}")

# Zip and download all saved results
print("\nPreparing download of saved results...")
!zip -r swinir_saved_results.zip {saved_results_folder}
files.download('swinir_saved_results.zip')

print("\nAll processing completed!")
print(f"Enhanced images have been saved to the '{saved_results_folder}' folder.")
print("The images are also available in the downloaded ZIP file.")